# PyTorch Fundamentals

This notebook closely follows the material available at learnpytorch.io [[1]](https://www.learnpytorch.io/) with occassional refactoring and extension for consistency of style and to make connections with other parts of the package. It is also more extensive on examples and does less revisit of lower level concepts once discussed.

### About PyTorch

PyTorch is an open source machine and/or, depending on accepted classification, deep learning framework with a highly optimized tensor workflow using both CPU and GPU for computation. It is widely used both in industry and academia, and, as of 2022, it is the most used deep learning framework on Papers with Code [[2]](https://paperswithcode.com/trends). Its design philosophy is based around the following principles.
- Usability over performance
- Simple over easy
- Python first with best in class language interoperability

### Install PyTorch and confirm version

The recommended way to install PyTorch in a virtual environment and CUDA 11.7 support is the following.

`pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117`

The version of `torch` references both the PyTorch version and the linked CUDA installation.

In [1]:
import torch
print(torch.__version__)

2.0.0+cu117


In order to check if there is an Nvidia GPU, we can run the `nvidia_smi` command on the command line. From within the notebook we can achieve the same by prefixing it with a bang `!`.

In [2]:
!nvidia-smi

Sun Mar 26 10:53:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti    WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   34C    P3               N/A /  N/A|      0MiB /  4096MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Note.** If the CUDA version listed differs from the one that shipped with the PyTorch binaries, then it may not necessarily be a problem. For instance, 11.7 version coming with PyTorch will be able to work with 12.1 on the GPU side. *This was tested with an NVIDIA GeForce GTX 1050 Ti 4GB GPU and driver version of 531.41.*

Cuda availability can be checked explicitly leaving it open to make the usage of the GPU conditional.

In [3]:
torch.cuda.is_available()

True

### Tensors

Tensors are higher-order generalizations of vector and matrix concepts and as such are implemented using the abstraction of higher-dimensional arrays. An object $T$ is called an $n$-order or $n$-dimensional tensor if there are vector spaces $V_i$ of dimensions $d_i$ for $i = 0, 1, \dots, n - 1$ such that

$$
T = V_0 \times V_1 \times \dots \times V_{n - 1}
$$

In PyTorch, the vector spaces have finitely many elements by nature (finiteness of representation) and can correspond to one of the supported data types.

In [4]:
torch_types = {
    "32-bit floating point": (torch.float32, torch.float),
    "64-bit floating point": (torch.float64, torch.double),
    "64-bit complex": (torch.complex64, torch.cfloat),
    "128-bit complex": (torch.complex128, torch.cdouble),
    "16-bit floating point (1 sign, 5 exponents, 10 significand )": (torch.float16, torch.half),
    "16-bit Brain floating point (1 sign, 8 exponent, 7 significand)": (torch.bfloat16,),
    "8-bit integer (unsigned)": (torch.uint8,),
    "8-bit integer (signed)": (torch.int8,),
    "16-bit integer (signed)": (torch.int16, torch.short),
    "32-bit integer (signed)": (torch.int32, torch.int),
    "64-bit integer (signed)": (torch.int64, torch.long),
    "Boolean": torch.bool,
}

Perhaps not surprisingly, standard nested `list` objects that respect the homogeneous shape requirements, can be used to construct the first `Tensor` examples.

In [5]:
vector = torch.tensor([1, 0])
matrix = torch.tensor(
    [
        [0., 1.],
        [1., 0.],
    ]
)
tensor = torch.tensor(
    [
        [
            [True, False],
            [False, True],
        ],
        [
            [True, True],
            [False, False],
        ]
    ]
)
for t in [vector, matrix, tensor]:
    print(f"{t}\n\n\ttensor of type {t.dtype}\n")

tensor([1, 0])

	tensor of type torch.int64

tensor([[0., 1.],
        [1., 0.]])

	tensor of type torch.float32

tensor([[[ True, False],
         [False,  True]],

        [[ True,  True],
         [False, False]]])

	tensor of type torch.bool



**Note.** Above examples also show the default `dtype` chosen for Python types.

PyTorch supports not-a-number values in the form of `float("nan")` and also positive and negative infinite values `float("Inf")` and `float("-Inf")`, respectively.
It worth mentioning that the Boolean type in PyTorch is not nulleable, NaN values are considered `True`.

In [6]:
torch.tensor([float(item) for item in ["nan", "Inf", "-Inf"]], dtype=torch.bool)

tensor([True, True, True])

In mathematics, the most straighforward explanation of a tensor would be a multilinear mapping. However, in PyTorch, tensors are meant to be understood more as real world inputs mapped onto supported data types, at least initially. For example, a 2 by 2 pixels image can be  considered as a tensor of order 3 over integer-type domains of dimensions 2, 2, and 3, where the first and second orders with dimension 2 each stand for the height and the width, respectively, and the third order of dimension 3 reflects the color channel (RGB).

In [7]:
image = torch.tensor(
    [
        [
            [0.1, 0.2, 0.3],  # RGB of pixel (0, 0)
            [0.2, 0.2, 0.3],  # RGB of pixel (0, 1)
        ],
        [
            [0.4, 0.5, 0.5],  # RGB of pixel (1, 0)
            [0.5, 0.6, 0.6],  # RGB of pixel (1, 1)
        ]
    ]
)
image.shape

torch.Size([2, 2, 3])

### Deterministic tensor constructors

Let us begin with the convention that scalars are thought of as tensors of dimension 0 rather than dimension 1 over any of the allowed domains.

In [7]:
scalar = torch.tensor(0)
print(f"{scalar} is an object {type(scalar)} of dimension {scalar.ndim}")

0 is an object <class 'torch.Tensor'> of dimension 0


We have already seen the conversion of Python types into tensors by means of `torch.tensor`. This is fairly inefficient due to the performance of Python objects and also rarely needed as under normal circumstances inputs are flowing in through I/O streams. We may additionally need to create tensors of special forms to supplement the flow of computation.

In [8]:
zeros = torch.zeros((2, 2))
print(zeros)

tensor([[0., 0.],
        [0., 0.]])


In [9]:
ones = torch.ones((3, 2, 1))
print(ones)

tensor([[[1.],
         [1.]],

        [[1.],
         [1.]],

        [[1.],
         [1.]]])


In [10]:
arith_range = torch.arange(30).reshape(2, 3, 5)
print(arith_range)

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]]])


In [11]:
equispaced = torch.linspace(0, 9, 10).reshape(2, 5)
print(equispaced)

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])


In [12]:
equiscaled = torch.logspace(0, 1, 10).reshape(2, 5)
print(equiscaled)

tensor([[ 1.0000,  1.2915,  1.6681,  2.1544,  2.7826],
        [ 3.5938,  4.6416,  5.9948,  7.7426, 10.0000]])


### Tensor algebra

Tensors generalize matrices not only in terms of representation, but also in terms of operations defined for them. PyTorch tensors support pointwise addition, subtraction, multiplication, and division. Also, as expected from vectors spaces, they support scalar multiplication, matrix multiplication along axes of identical dimensions, and also implement the transpose operation.

In [14]:
zeros = torch.zeros((2, 3))
ones = torch.ones((3, 2))
transponent = ones.t()
print(f"Transponent of\n{ones}\n  is:\n{transponent}")
print()
print(f"Sum of\n{zeros}\n  and\n{transponent}\n  is:\n{zeros + transponent}")
print()
print(f"Difference of\n{zeros}\n  and\n{transponent}\n  is:\n{zeros - transponent}")
print()
print(f"Pointwise product of\n{zeros}\n  and\n{transponent}\n  is:\n{zeros * transponent}")
print()
print(f"Pointwise quotient of\n{zeros}\n  and\n{transponent}\n  is:\n{zeros / transponent}")
print()
print(f"Double of\n{ones}\n  is:\n{2 * ones}")
print()
print(f"Matrix product of\n{zeros}\n  and\n{ones}\n  is:\n{torch.matmul(zeros, ones)})")

Transponent of
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
  is:
tensor([[1., 1., 1.],
        [1., 1., 1.]])

Sum of
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  and
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  is:
tensor([[1., 1., 1.],
        [1., 1., 1.]])

Difference of
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  and
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  is:
tensor([[-1., -1., -1.],
        [-1., -1., -1.]])

Pointwise product of
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  and
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  is:
tensor([[0., 0., 0.],
        [0., 0., 0.]])

Pointwise quotient of
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  and
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  is:
tensor([[0., 0., 0.],
        [0., 0., 0.]])

Double of
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
  is:
tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])

Matrix product of
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  and
tensor([[1., 1.],
  

Basic concepts of determinant and matrix inversion are also lifted to their higher dimensional analogues.

In [15]:
tensor = torch.linspace(0, 1, 16).reshape(2, 2, 2, 2)
print(f"(Hyper)determinant of\n{tensor}\n  is:\n{tensor.det()}")
print()
print(f"Inverse of\n{tensor}\n  is:\n{tensor.inverse()}")

(Hyper)determinant of
tensor([[[[0.0000, 0.0667],
          [0.1333, 0.2000]],

         [[0.2667, 0.3333],
          [0.4000, 0.4667]]],


        [[[0.5333, 0.6000],
          [0.6667, 0.7333]],

         [[0.8000, 0.8667],
          [0.9333, 1.0000]]]])
  is:
tensor([[-0.0089, -0.0089],
        [-0.0089, -0.0089]])

Inverse of
tensor([[[[0.0000, 0.0667],
          [0.1333, 0.2000]],

         [[0.2667, 0.3333],
          [0.4000, 0.4667]]],


        [[[0.5333, 0.6000],
          [0.6667, 0.7333]],

         [[0.8000, 0.8667],
          [0.9333, 1.0000]]]])
  is:
tensor([[[[ -22.5000,    7.5000],
          [  15.0000,    0.0000]],

         [[ -52.5000,   37.5000],
          [  45.0000,  -30.0000]]],


        [[[ -82.5003,   67.5003],
          [  75.0003,  -60.0002]],

         [[-112.4994,   97.4994],
          [ 104.9994,  -89.9995]]]])


**Note.** A legitim question at this point is that *"under what circumstances the determinant and inverse can be thought of as useful measures of input data?"*. In fact, these become more relevant once the tensor is thought of as a multilinear mapping or in an overly simplified manner as a state transition expression. The determinant helps understanding enlargening/shrinking effects while the inverse can be considered a reversal of transition.

Comparison is understood pontwise between tensors of identical shape. It is also possible to compare against scalars.

In [16]:
print(f"Positions where\n{zeros}\n  is less than\n{transponent}\n  are:\n{zeros < transponent}")
print()
print(f"Positions where\n{zeros}\n  is greater than\n{transponent}\n  are:\n{zeros > transponent}")
print()
print(f"Positions where\n{zeros}\n  is equal to\n{transponent}\n  are:\n{zeros == transponent}")
print()
print(f"Positions where\n{zeros}\n  is not equal to\n{transponent}\n  are:\n{zeros != transponent}")
print()

Positions where
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  is less than
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  are:
tensor([[True, True, True],
        [True, True, True]])

Positions where
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  is greater than
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  are:
tensor([[False, False, False],
        [False, False, False]])

Positions where
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  is equal to
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  are:
tensor([[False, False, False],
        [False, False, False]])

Positions where
tensor([[0., 0., 0.],
        [0., 0., 0.]])
  is not equal to
tensor([[1., 1., 1.],
        [1., 1., 1.]])
  are:
tensor([[True, True, True],
        [True, True, True]])



Boolean operations apply pointwise too; the boolean value of tensors themselves are ambiguous.

In [17]:
tensor = torch.arange(9).reshape(3, 3)
print(f"Positions where\n{tensor}\n  is less than 3 or larger than 6 are\n{(tensor < 3) | (tensor > 6)}")
print()
print(f"Positions where\n{tensor}\n  is less than 6 and larger than 3 are\n{(tensor > 3) & (tensor < 6)}")
print()
print(f"Positions where\n{tensor}\n  is less than or equal to 3 or larger than or equal to 6\n{~((tensor > 3) & (tensor < 6))}")

Positions where
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
  is less than 3 or larger than 6 are
tensor([[ True,  True,  True],
        [False, False, False],
        [False,  True,  True]])

Positions where
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
  is less than 6 and larger than 3 are
tensor([[False, False, False],
        [False,  True,  True],
        [False, False, False]])

Positions where
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
  is less than or equal to 3 or larger than or equal to 6
tensor([[ True,  True,  True],
        [ True, False, False],
        [ True,  True,  True]])


### Stochastic tensor constructors

Often times, we need to sample values of a tensor from some probability distribution. PyTorch offers stochastic constructors for many common distributions.

In [18]:
uniform_tensor = torch.rand(2, 3, 4)
print(uniform_tensor)

tensor([[[0.0343, 0.0736, 0.4920, 0.2369],
         [0.8754, 0.5869, 0.9185, 0.4064],
         [0.0527, 0.5695, 0.3841, 0.9783]],

        [[0.4508, 0.6463, 0.4456, 0.4547],
         [0.2985, 0.6598, 0.8246, 0.0016],
         [0.9147, 0.9897, 0.2439, 0.5534]]])


In [19]:
# Unconventional constructor using an pointwise free p parameter
# It does have to do with Markov-chains and evolutionary processes
bernoulli_tensor = torch.bernoulli(
    0.5 * torch.ones((10, 10))  # Fix p to be 0.5 across all the sampling
)
print(bernoulli_tensor)

tensor([[0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 1., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
        [1., 0., 0., 1., 1., 1., 1., 0., 1., 0.],
        [0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 1., 0., 1., 0., 0., 0., 1.],
        [1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]])


In [20]:
gaussian_tensor = torch.normal(0, 1, (5, 5))
print(gaussian_tensor)

tensor([[ 0.3549, -1.1250, -0.5727,  0.8278, -1.5543],
        [ 0.7146,  0.4246,  0.0951, -2.0643,  0.6043],
        [-0.6418,  0.1732,  1.7667, -0.9364,  1.6831],
        [ 0.1733,  0.0926,  0.2587,  0.3330,  0.9125],
        [ 0.7031, -0.7644, -0.3961,  1.0228,  0.6673]])


In [21]:
# Same comment applies as for Bernoulli sampling
poisson_tensor = torch.poisson(
    torch.ones((5, 5))  # Fix lambda to be 1 across all the sampling
)
print(poisson_tensor)

tensor([[1., 1., 3., 0., 0.],
        [2., 2., 1., 0., 1.],
        [1., 0., 1., 0., 2.],
        [1., 1., 1., 0., 0.],
        [1., 1., 0., 1., 2.]])


In case sampling scenarios are to be repeated, the random seed can be fixed to ensure identical outputs. Note that the seed moves away during each sampling according to the generator process and needs to be reset.

In [22]:
def reset_seed():
    torch.manual_seed(seed=1)

reset_seed()
print(torch.rand(3, 3))
reset_seed()
print(torch.rand(3, 3))
reset_seed()
print(torch.rand(3, 3))

tensor([[0.7576, 0.2793, 0.4031],
        [0.7347, 0.0293, 0.7999],
        [0.3971, 0.7544, 0.5695]])
tensor([[0.7576, 0.2793, 0.4031],
        [0.7347, 0.0293, 0.7999],
        [0.3971, 0.7544, 0.5695]])
tensor([[0.7576, 0.2793, 0.4031],
        [0.7347, 0.0293, 0.7999],
        [0.3971, 0.7544, 0.5695]])


### Indexing

Tensors implement the standard accessors by position: explicit indexing in one or multiple dimensions and slicing.

In [50]:
tensor = torch.arange(24).reshape(2, 3, 4)
print(f"Tensor example used is:\n{tensor}\nWe are going to think along x, y, and z axes respectively.")
print()
print(f"Accessing first element along axis x:\n{tensor[0]}")
print()
print(f"Accessing first element along axis y:\n{tensor[:, 0]}")
print()
print(f"Accessing first element along axis z:\n{tensor[:, :, 0]}")
print()
print(f"Accessing first element along axis x and y:\n{tensor[0, 0]}")
print()
print(f"Accessing first element along axis x and z:\n{tensor[0, :, 0]}")
print()
print(f"Accessing first element along axis y and z:\n{tensor[:, 0, 0]}")
print()
print(f"Accessing first element along axis x, second and third along y:\n{tensor[0, 1:]}")

Tensor example used is:
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
We are going to think along x, y, and z axes respectively.

Accessing first element along axis x:
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

Accessing first element along axis y:
tensor([[ 0,  1,  2,  3],
        [12, 13, 14, 15]])

Accessing first element along axis z:
tensor([[ 0,  4,  8],
        [12, 16, 20]])

Accessing first element along axis x and y:
tensor([0, 1, 2, 3])

Accessing first element along axis x and z:
tensor([0, 4, 8])

Accessing first element along axis y and z:
tensor([ 0, 12])

Accessing first element along axis x, second and third along y:
tensor([[ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


In addition, tensors support boolean masking like NumPy arrays and Pandas dataframes.

In [55]:
tensor = torch.normal(0, 1, (3, 3, 3))
print(f"Tensor example used is:\n{tensor}")
print()
print(f"Elements less than 0 are:\n{tensor[tensor < 0]}")
print()
print(f"Elements between -1 and 1:\n{tensor[(-1 < tensor) & (tensor < 1)]}")

Tensor example used is:
tensor([[[-0.5007,  0.1164, -0.7695],
         [ 2.2969, -0.2332,  0.4124],
         [-1.2673,  1.5329,  1.4025]],

        [[ 0.8013, -0.5650, -0.9675],
         [-1.8558,  0.3900,  0.8584],
         [ 0.6327, -0.0534,  0.1181]],

        [[ 1.3405, -0.3490, -0.4198],
         [ 1.0553, -0.6366,  1.6225],
         [ 0.2911,  0.0630,  0.0171]]])

Elements less than 0 are:
tensor([-0.5007, -0.7695, -0.2332, -1.2673, -0.5650, -0.9675, -1.8558, -0.0534,
        -0.3490, -0.4198, -0.6366])

Elements between -1 and 1:
tensor([-0.5007,  0.1164, -0.7695, -0.2332,  0.4124,  0.8013, -0.5650, -0.9675,
         0.3900,  0.8584,  0.6327, -0.0534,  0.1181, -0.3490, -0.4198, -0.6366,
         0.2911,  0.0630,  0.0171])


### Aggregation

Since tensors are used for representation of input data, in many scenarios, we are interested in aggregate measures, more precisely statistics, along various axes. Most common descriptives are implemented as methods of `Tensor` objects. Note that some aggregates return a corresponding type that is more of a view with a tensor of indices supplementing it (for instance, minimum, maximum, median, etc...).

In [49]:
sample = torch.normal(0, 1, (5, 5, 5))

print(f"Tensor example used is:\n{sample}")
print()
print(f"Mean across all three axes is:\n{sample.mean()}")
print()
print(f"Means across x axis are:\n{sample.mean(0)}")
print()
print(f"Maximums across y axis are:\n{sample.max((1))}")
print()
print(f"Minimums across z axis are:\n{sample.min(2)}")
print()
print(f"Standard deviations across x and y axes are:\n{sample.std((0, 1))}")
print()
print(f"Medians across x and z axes are:\n{sample.median((0))[0].median(1)}")
print()
print(f"Cummulative sums across y and z axes are:\n{sample.cumsum(1)[0].cumsum(1)}")
print()

Tensor example used is:
tensor([[[-1.1509e+00,  9.6896e-01, -2.6251e+00,  1.9025e+00,  2.8895e-01],
         [ 1.1787e+00,  1.0630e+00, -4.8261e-01, -3.8114e-01, -8.3449e-02],
         [-4.4002e-01,  6.6888e-01, -1.8674e+00, -3.9340e-01, -5.9160e-01],
         [-7.2037e-02, -1.2035e+00, -1.3180e+00,  2.4631e-01,  7.9623e-01],
         [ 1.2749e+00,  1.5626e-01,  2.4801e+00,  5.9284e-01,  5.9499e-01]],

        [[-4.8542e-02,  7.9916e-01, -6.9276e-01, -1.7733e+00,  1.2558e+00],
         [-9.7308e-01, -1.2769e-01, -1.7656e-01,  7.2515e-01, -3.1347e+00],
         [ 6.2105e-01, -1.0342e+00, -5.6700e-01, -8.3042e-01, -1.3737e-01],
         [-6.5589e-01, -8.9030e-01,  1.7062e+00,  8.0476e-04, -1.6036e-01],
         [ 1.4063e-01, -1.8038e+00, -1.3119e+00, -7.3711e-01, -8.1172e-01]],

        [[ 1.8271e+00, -1.1269e+00, -1.3796e+00,  1.2199e+00,  1.6074e+00],
         [ 5.3705e-01, -1.8889e-01, -2.5139e-01,  1.9611e-01, -8.8677e-01],
         [ 5.7864e-02, -1.0375e+00,  2.3136e-01, -1.0128e-01

### CPU vs. GPU

When a GPU is available, then ideally, we should be able to leverage it for computation. This requires the tensor to be stored on the GPU device rather than the CPU which is the default.

In [56]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [60]:
tensor_on_cpu = torch.rand(3, 3)
tensor_on_gpu = torch.rand(3, 3, device=DEVICE)
print(f"Tensor\n{tensor_on_cpu}\n  is stored on {tensor_on_cpu.device}.")
print()
print(f"Tensor\n{tensor_on_cpu}\n  is stored on {tensor_on_gpu.device}.")
print()
print(f"We can also move between the devices with `.to(<device>)`tensor_on_cpu.to(DEVICE).device

Tensor
tensor([[0.5569, 0.7303, 0.7923],
        [0.9384, 0.1551, 0.6396],
        [0.0666, 0.6861, 0.2434]])
  is stored on cpu.

Tensor
tensor([[0.5569, 0.7303, 0.7923],
        [0.9384, 0.1551, 0.6396],
        [0.0666, 0.6861, 0.2434]])
  is stored on cuda:0.



device(type='cuda', index=0)

### References

[1] Learn PyTorch for Deep Learning: Zero to Mastery book, accessed online on 2023.03.25.

[2] Papers with Code trends, accessed online on 2023.03.25.